<a href="https://colab.research.google.com/github/mezaga/HugoProyect/blob/main/analisis_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive') 

Mounted at /content/drive


In [ ]:
!pip install essentia

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 13.6 MB 26.3 MB/s 


In [ ]:
import essentia

# there are two operating modes in essentia which (mostly) have the same algorithms
# they are accessible via two submodules:
import essentia.standard as es
import essentia.streaming

# let's have a look at what is in there
print(dir(essentia.standard))

['AfterMaxToBeforeMaxEnergyRatio', 'AllPass', 'AudioLoader', 'AudioOnsetsMarker', 'AudioWriter', 'AutoCorrelation', 'BFCC', 'BPF', 'BandPass', 'BandReject', 'BarkBands', 'BeatTrackerDegara', 'BeatTrackerMultiFeature', 'Beatogram', 'BeatsLoudness', 'BinaryOperator', 'BinaryOperatorStream', 'BpmHistogram', 'BpmHistogramDescriptors', 'BpmRubato', 'CartesianToPolar', 'CentralMoments', 'Centroid', 'ChordsDescriptors', 'ChordsDetection', 'ChordsDetectionBeats', 'ChromaCrossSimilarity', 'Chromagram', 'Chromaprinter', 'ClickDetector', 'Clipper', 'ConstantQ', 'CoverSongSimilarity', 'Crest', 'CrossCorrelation', 'CrossSimilarityMatrix', 'CubicSpline', 'DCRemoval', 'DCT', 'Danceability', 'Decrease', 'Derivative', 'DerivativeSFX', 'DiscontinuityDetector', 'Dissonance', 'DistributionShape', 'Duration', 'DynamicComplexity', 'ERBBands', 'EasyLoader', 'EffectiveDuration', 'Energy', 'EnergyBand', 'EnergyBandRatio', 'Entropy', 'Envelope', 'EqloudLoader', 'EqualLoudness', 'Extractor', 'FFT', 'FFTC', 'Fade

In [ ]:


audiofile = '/content/drive/MyDrive/UNAM/audioLibrosa/test01.mp3'

# Compute all features.
# Aggregate 'mean' and 'stdev' statistics for all low-level, rhythm, and tonal frame features.
features, features_frames = es.MusicExtractor(lowlevelStats=['mean', 'stdev'],
                                              rhythmStats=['mean', 'stdev'],
                                              tonalStats=['mean', 'stdev'])(audiofile)

# See all feature names in the pool in a sorted order
print(sorted(features.descriptorNames()))

['lowlevel.average_loudness', 'lowlevel.barkbands.mean', 'lowlevel.barkbands.stdev', 'lowlevel.barkbands_crest.mean', 'lowlevel.barkbands_crest.stdev', 'lowlevel.barkbands_flatness_db.mean', 'lowlevel.barkbands_flatness_db.stdev', 'lowlevel.barkbands_kurtosis.mean', 'lowlevel.barkbands_kurtosis.stdev', 'lowlevel.barkbands_skewness.mean', 'lowlevel.barkbands_skewness.stdev', 'lowlevel.barkbands_spread.mean', 'lowlevel.barkbands_spread.stdev', 'lowlevel.dissonance.mean', 'lowlevel.dissonance.stdev', 'lowlevel.dynamic_complexity', 'lowlevel.erbbands.mean', 'lowlevel.erbbands.stdev', 'lowlevel.erbbands_crest.mean', 'lowlevel.erbbands_crest.stdev', 'lowlevel.erbbands_flatness_db.mean', 'lowlevel.erbbands_flatness_db.stdev', 'lowlevel.erbbands_kurtosis.mean', 'lowlevel.erbbands_kurtosis.stdev', 'lowlevel.erbbands_skewness.mean', 'lowlevel.erbbands_skewness.stdev', 'lowlevel.erbbands_spread.mean', 'lowlevel.erbbands_spread.stdev', 'lowlevel.gfcc.cov', 'lowlevel.gfcc.icov', 'lowlevel.gfcc.mean

In [ ]:
print(features['lowlevel.mfcc.mean'])

[-655.4984      99.37667    -20.99133     36.590763    34.14451
   15.775821    22.551374    16.001038    -2.882886    -2.2176323
   -1.5414951    6.2058835   -2.9418406]


In [ ]:
# Compute beat positions and BPM.
rhythm_extractor = es.RhythmExtractor2013(method="multifeature")
bpm, beats, beats_confidence, _, beats_intervals = rhythm_extractor(audio)

In [ ]:
audio = es.MonoLoader(filename='/content/drive/MyDrive/UNAM/audioLibrosa/test01.mp3')()
lowlevel = es.LowLevelSpectralExtractor()
result = lowlevel(audio)

In [ ]:
audio = es.MonoLoader(filename='/content/drive/MyDrive/UNAM/audioLibrosa/salida2.wav')()
lowlevel = es.LowLevelSpectralExtractor()
result = lowlevel(audio)

In [ ]:
for one in result:
  print("NUEMVO")
  print(len(one))

In [ ]:
# Load audio file.
audio = es.MonoLoader(filename='/content/drive/MyDrive/UNAM/audioLibrosa/salida2.wav')()

# 1. Compute the onset detection function (ODF).

# The OnsetDetection algorithm provides various ODFs.
od_hfc = es.OnsetDetection(method='hfc')
od_complex = es.OnsetDetection(method='complex')

# We need the auxilary algorithms to compute magnitude and phase.
w = es.Windowing(type='hann')
fft = es.FFT() # Outputs a complex FFT vector.
c2p = es.CartesianToPolar() # Converts it into a pair of magnitude and phase vectors.

# Compute both ODF frame by frame. Store results to a Pool.
pool = essentia.Pool()
for frame in es.FrameGenerator(audio, frameSize=1024, hopSize=512):
    magnitude, phase = c2p(fft(w(frame)))
    pool.add('odf.hfc', od_hfc(magnitude, phase))
    pool.add('odf.complex', od_complex(magnitude, phase))

# 2. Detect onset locations.
onsets = es.Onsets()

onsets_hfc = onsets(# This algorithm expects a matrix, not a vector.
                    essentia.array([pool['odf.hfc']]),
                    # You need to specify weights, but if we use only one ODF
                    # it doesn't actually matter which weight to give it
                    [1])

onsets_complex = onsets(essentia.array([pool['odf.complex']]), [1])

print(onsets_hfc)

[ 0.12770975  0.63854873  0.8126984   1.9272562   2.1478457   2.4264853
  2.554195    2.9141042   3.2856236   3.7035828   3.8777325   4.144762
  4.3189116   4.7136507   4.783311    5.2128797   5.4218593   5.619229
  5.8630385   5.8978686   6.0023584   6.072018    6.188118    6.304218
  6.3738775   6.5248075   6.6292973   6.722177    6.9311566   7.1401362
  7.3723354   7.476825    7.569705    7.7902946   7.8947845   7.9992743
  8.196644    8.4288435   8.846803    9.090612    9.206712    9.496962
  9.717551    9.856871    9.926531   10.170341   10.27483    10.3561
 10.57669    10.808888   11.006259   11.215239   11.435827   11.644807
 11.853786   11.993107   12.074376   12.283356  ]
